In [1]:
import pandas as pd
# song_input = 'music_data_sample.txt'
# list_input = 'list_data_out_sample.txt'
song_input = 'Musi
c_Data_Out.txt'
list_input = 'List_Data_Out.txt'

def read_surprise_file(input_path, delimiter):
    return pd.read_table(input_path, delimiter=delimiter)

In [2]:
song = read_surprise_file(song_input, "--")
album = read_surprise_file(list_input, "--")

print(song.head)
print(album.head)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


<bound method NDFrame.head of                                                 song_name  \
0                              Shelly - (日剧《夏洛克：未叙之章》主题曲)   
1                                       FULLMETAL TRIGGER   
2                     Knock U down - (TBS《CDTV》12・1月份开场曲)   
3                  Don't (LIVE:2019/6/18 @Zepp DiverCity)   
4       愛のために ～for love, for a child～ - (为了爱～for love,...   
...                                                   ...   
107209            じゃすと・べりー・くいっくりー! - (Just Very Quickly!)   
107210  悠久のカタルシス ~ラフィー ver.~ - (Everlasting Catharsis ...   
107211             スリーピング・ワンダーランド - (Sleeping Wonderland)   
107212  悠久のカタルシス ~エンタープライズ ver.~ - (Everlasting Cathar...   
107213                               Pledge of liberation   

                              singer     song_id    album_id  
0                       DEAN FUJIOKA  1406131972  2819914042  
1       THE RAMPAGE from EXILE TRIBE  1408601639  2819914042  
2                               水樹奈々  1407987266

In [3]:
import pickle
song_dict = {}
song_out_path = "songs_dict.pkl"
for i in range(song.shape[0]):
    song_dict[song.iloc[i, 2]] = song.iloc[i, 0]
pickle.dump(song_dict, open(song_out_path,"wb"))

In [4]:
# 生成字典
album_id2name = {}
album_name2id = {}
album_id2name_out = "album_id2name.pkl"
album_name2id_out = "album_name2id.pkl"
for i in range(album.shape[0]):
    album_id2name[album.iloc[i, 1]] = album.iloc[i, 0]
    album_name2id[album.iloc[i, 0]] = album.iloc[i, 1]
pickle.dump(album_id2name, open(album_id2name_out,"wb"))
pickle.dump(album_name2id, open(album_name2id_out,"wb"))

In [5]:
song_id2name = pickle.load(open(song_out_path,'rb'))
song_id2name[1407987266]

'Knock\xa0U\xa0down - (TBS《CDTV》12・1月份开场曲)'

In [6]:
album_id2name = pickle.load(open(album_id2name_out,'rb'))
album_id2name[2750999763]

'遗世独立•旷远山谷的喧嚣摆脱'

In [7]:
album_name2id = pickle.load(open(album_name2id_out,'rb'))
album_name2id['遗世独立•旷远山谷的喧嚣摆脱']

2750999763

In [8]:
song['rating']=1
song['timestamp']=130000000

In [9]:
surprise_data = song.drop(song.columns[0:2], axis=1)
album_id = surprise_data.pop('album_id')
surprise_data.insert(0, 'album_id', album_id)
print(surprise_data)

          album_id     song_id  rating  timestamp
0       2819914042  1406131972       1  130000000
1       2819914042  1408601639       1  130000000
2       2819914042  1407987266       1  130000000
3       2819914042  1407245276       1  130000000
4       2819914042  1408605111       1  130000000
...            ...         ...     ...        ...
107209  3045620722  1398242388       1  130000000
107210  3045620722  1395784890       1  130000000
107211  3045620722  1395784893       1  130000000
107212  3045620722  1393152305       1  130000000
107213  3045620722  1393152304       1  130000000

[107214 rows x 4 columns]


In [10]:
list(surprise_data)

['album_id', 'song_id', 'rating', 'timestamp']

In [11]:
data_path = "music.csv"
surprise_data.to_csv(data_path, index=False, header=False)

In [18]:
from surprise import SVD, KNNBaseline, KNNBasic
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate

In [19]:
print("开始训练模型...")
reader = Reader(line_format='user item rating timestamp', sep=',')
data = Dataset.load_from_file(data_path, reader=reader)

algo = KNNBaseline()
# perf = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5)
all_trainset = data.build_full_trainset()
algo.fit(all_trainset)
#输出结果
print(algo.trainset.to_inner_uid('2750999763'))

开始训练模型...
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
1


In [20]:
current_playlist = '<心动计划>触动你心跳加速的旋律'
print("歌单名称:", current_playlist)

#歌单名称转歌单id
album_id = album_name2id[current_playlist]
print("歌单id:",album_id)

#歌单id转surprise内部user id
album_inner_id = algo.trainset.to_inner_uid(str(album_id))
print("内部id:",album_inner_id)

歌单名称: <心动计划>触动你心跳加速的旋律
歌单id: 2946956241
内部id: 49


In [21]:
#根据内部id找邻近值,返回邻近的内部user id
album_neighbors = algo.get_neighbors(album_inner_id,k=10)

#把内部userid转成歌单id,再转换成歌单名称返还
album_neighbors = (algo.trainset.to_raw_uid(inner_id) for inner_id in album_neighbors)
album_neighbors = (album_id2name[int(album_id)] for album_id in album_neighbors)

print("和歌单 《", current_playlist, "》 最相似的10个歌单为：\n")
for album in album_neighbors:
    print(album, album_name2id[album])
#     print("内部uid:", algo.trainset.to_inner_uid(str(album_name2id[album])))

和歌单 《 <心动计划>触动你心跳加速的旋律 》 最相似的10个歌单为：

清新小调 | 来自冬日漫步里的缱绻浪漫 3107590676
2010年代Billboard十年单曲榜TOP100 3054732286
♡SloMo宝藏BGM♡ 2892787946
有事上车再说. 2881720365
打游戏必备【战歌、节奏电音、重金属】 2816437068
秋风一起，连不小心打声喷嚏都是“爱秋！” 2995781805
李信BGM 『游戏专用』 3039249421
耳朵上的风景，电影里的乡村与民谣 2980709400
在甜橙和淡茶的交汇中 阳光都变得温柔起来 2945719972
流汗吧健身少女 2876709606


In [22]:
item_algo = KNNBasic(k=40,min_k=3,sim_options={'user_based': False}) # sim_options={'name': 'cosine','user_based': True} cosine/msd/pearson/pearson_baseline
item_algo.fit(all_trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [23]:
def get_similar_songs(top_k, song_id):
    item_inner_id = item_algo.trainset.to_inner_iid(song_id)
    item_neighbors = item_algo.get_neighbors(item_inner_id, k=top_k)
    f_item_neighbors = (item_algo.trainset.to_raw_iid(inner_id)
                       for inner_id in item_neighbors)
    return f_item_neighbors

current_song_id = 1406131972
print("和歌曲 《", song_id2name[1406131972], "》 最相似的10首歌曲为：\n")
for song_id in list(get_similar_songs(10,'1406131972')):
    print(song_id2name[int(song_id)])

和歌曲 《 Shelly - (日剧《夏洛克：未叙之章》主题曲) 》 最相似的10首歌曲为：

FULLMETAL TRIGGER
Knock U down - (TBS《CDTV》12・1月份开场曲)
Don't (LIVE:2019/6/18 @Zepp DiverCity)
愛のために ～for love, for a child～ - (为了爱～for love, for a child～)
雨上がり
グリム - (Grimm)
Higher Dimension
コンプレックス・シューティング - (Complex Shooting)
new life
悠久のカタルシス ~クリーブランド ver.~ - (Everlasting Catharsis ~Cleveland ver.~)
